# Creating a Deploy App for Multi-Ai with Multiple Models

This tutorial shows how to create an inference application with multiple models, focusing on model files organization, inferring with named model in the application, and packaging.

Typically multiple models will work in tandem, e.g. a lung segmentation model's output, along with the original image, are be used by a lung nodule detection and classification model. However, there are currently no such models in the [MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo), so two independent models will be used in this example, [Spleen Segmentation](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation) and [Pancreas Segmentation](https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation), both are trained with DICOM images of CT modality, and both are packaged in the [MONAI Bundle](https://docs.monai.io/en/latest/bundle_intro.html) format. A single input of a CT Abdomen DICOM Series can be used for both models within the application.


## Important Steps
- Place the model TorchScripts in a defined folder structure, see below for details
- Pass the model name to the inference operator instance in the app
- Connect the input to and output from the inference operators, as required by the app

## Required Model File Organization

- The model files in TorchScript, be it MONAI Bundle compliant or not, must each be placed in an uniquely named folder. The name of this folder is used within the application to retrieve the loaded model network from the execution context.
- The folders containing the individual model file must then be place under a parent folder. The name of this folder can be any valid folder name chosen by the application developer.

## Example Model File Organization

In this example, the models are organized as shown below.
```
multi_models
├── pancreas_ct_dints
│   └── model.ts
└── spleen_ct
    └── model.ts
```

Please note,

- The `multi_models` is parent folder, and its path will be used as the model path when using App SDK CLI `exec` and `package` commands.
- The sub-folder names become model names, `pancreas_ct_dints` and `spleen_model`, respectively.

In the following sections, we will demonstrate how to create and package the application using these two models.

:::{note}
The two models are both MONAI bundles, published in [MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo)
- [spleen_ct_segmentation, v0.3.2](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation)
- [pancreas_ct_dints_segmentation, v0.3.0](https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation)

The DICOM CT series used as test input is downloaded from [TCIA](https://www.cancerimagingarchive.net/), CT Abdomen Collection ID `CPTAC-PDA` Subject ID `C3N-00198`.

Both the DICOM files and the models have been packaged and shared on Google Drive.
:::

## Creating Operators and connecting them in Application class

We will implement an application that consists of seven Operators:

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: a folder path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
    - **Input(selection_rules)**: a selection rule (Dict)
    - **Output(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **MonaiBundleInferenceOperator** x 2:
    - **Input(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output(pred)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **DICOMSegmentationWriterOperator** x2:
    - **Input(seg_image)**: a segmentation image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(dicom_seg_instance)**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))


:::{note}
The `DICOMSegmentationWriterOperator` needs both the segmentation image as well as the original DICOM series meta-data in order to use the patient demographics and the DICOM Study level attributes.
:::

The workflow of the application is illustrated below.

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB
    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list

    DICOMSeriesToVolumeOperator --|> Spleen_BundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> Spleen_DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    Spleen_BundleInferenceOperator --|> Spleen_DICOMSegmentationWriterOperator : pred...seg_image

    DICOMSeriesToVolumeOperator --|> Pancreas_BundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> Pancreas_DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    Pancreas_BundleInferenceOperator --|> Pancreas_DICOMSegmentationWriterOperator : pred...seg_image

    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class Spleen_BundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class Pancreas_BundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class Spleen_DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
    class Pancreas_DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### Setup environment


In [1]:
# Install MONAI and other necessary image processing packages for the application
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.12.0"
!python -c "import numpy" || pip install -q "numpy>=1.21"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard~=2.12.1"

# Install MONAI Deploy App SDK package
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

Note: you may need to restart the Jupyter kernel to use the updated packages.

### Download/Extract input and model/bundle files from Google Drive

In [2]:
# Download the test data and MONAI bundle zip file
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1llJ4NGNTjY187RLX4MtlmHYhfGxBNWmd"

# After downloading ai_spleen_bundle_data zip file from the web browser or using gdown,
!unzip -o "ai_multi_model_bundle_data.zip"

Downloading...
From (uriginal): https://drive.google.com/uc?id=1llJ4NGNTjY187RLX4MtlmHYhfGxBNWmd
From (redirected): https://drive.google.com/uc?id=1llJ4NGNTjY187RLX4MtlmHYhfGxBNWmd&confirm=t&uuid=a4ff9f27-be83-4002-bc3b-8817adc3026e
To: /home/mqin/src/monai-deploy-app-sdk/notebooks/tutorials/ai_multi_model_bundle_data.zip
100%|████████████████████████████████████████| 647M/647M [00:08<00:00, 72.5MB/s]
Archive:  ai_multi_model_bundle_data.zip
  inflating: dcm/1-001.dcm           
  inflating: dcm/1-002.dcm           
  inflating: dcm/1-003.dcm           
  inflating: dcm/1-004.dcm           
  inflating: dcm/1-005.dcm           
  inflating: dcm/1-006.dcm           
  inflating: dcm/1-007.dcm           
  inflating: dcm/1-008.dcm           
  inflating: dcm/1-009.dcm           
  inflating: dcm/1-010.dcm           
  inflating: dcm/1-011.dcm           
  inflating: dcm/1-012.dcm           
  inflating: dcm/1-013.dcm           
  inflating: dcm/1-014.dcm           
  inflating: dcm/1-015

### Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [3]:
import logging

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

import monai.deploy.core as md
from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)


### Determining the Model Name and IOfor the Model Bundle Inference Operator

The App SDK provides a `MonaiBundleInferenceOperator` class to perform inference with a MONAI Bundle, which is essentially a PyTorch model in TorchScript with additional metadata describing the model network and processing specification. This operator uses the MONAI utilities to parse a MONAI Bundle to automatically instantiate the objects required for input and output processing as well as inference, as such it depends on MONAI transforms, inferers, and in turn their dependencies.

Each Operator class inherits from the base [Operator](/modules/_autosummary/monai.deploy.core.Operator) class, and its input/output properties are specified by using [@input](/modules/_autosummary/monai.deploy.core.input)/[@output](/modules/_autosummary/monai.deploy.core.output) decorators. For the `MonaiBundleInferenceOperator` class, the input/output need to be defined to match those of the model network, both in name and data type. For the current release, an `IOMapping` object is used to connect the operator input/output to those of the model network by using the same names. This is likely to change, to be automated, in the future releases once certain limitation in the App SDK is removed.

When multiple models are used in an application, it is important that each model network name is passed in when creating the inference operator, via the `model_name` argument.

The Spleen CT Segmentation and Pancreas model networks have a named input, called "image", and the named output called "pred", and both are of image type. These can all be mapped to the App SDK [Image](/modules/_autosummary/monai.deploy.core.domain.Image). This piece of information is typically acquired by examining the model metadata `network_data_format` attribute in the bundle, as seen in this [example] (https://github.com/Project-MONAI/model-zoo/blob/dev/models/spleen_ct_segmentation/configs/metadata.json).

### Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](/modules/_autosummary/monai.deploy.core.Application) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](/modules/_autosummary/monai.deploy.core.resource) and [@env](/modules/_autosummary/monai.deploy.core.env) decorators.

Objects required for DICOM parsing, series selection, pixel data conversion to volume image, model specific inference, and the AI result specific DICOM Segmentation object writers are created. The execution pipeline, as a Directed Acyclic Graph, is then constructed by connecting these objects through <a href="../../modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

In [4]:
@resource(cpu=1, gpu=1, memory="7Gi")
# Enforcing torch>=1.12.0 because one of the Bundles/TorchScripts, Pancreas CT Seg, was created
# with this version, and would fail to jit.load with lower version of torch.
# The Bundle Inference Operator as of now only requires torch>=1.10.2, and does not yet dynamically
# parse the MONAI bundle to get the required pip package or ver on initialization, hence it does not set
# its own @env decorator accordingly when the app is being packaged into a MONAI Package.
@md.env(pip_packages=["torch>=1.12.0"])
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
# The monai pkg is not required by this class, instead by the included operators.
class App(Application):
    """This example demonstrates how to create a multi-model/multi-AI application.

    The important steps are:
        1. Place the model TorchScripts in a defined folder structure, see below for details
        2. Pass the model name to the inference operator instance in the app
        3. Connect the input to and output from the inference operators, as required by the app

    Model organization for this example:

        multi_models
        ├── pancreas_ct_dints
        │   └── model.ts
        └── spleen_ct
            └── model.ts
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init to provide the optional packages info, parsed from the bundle, to the packager
        # for it to install the packages in the MAP docker image.
        # Setting output IOType to DISK only works only for leaf operators, not the case in this example.
        # When multiple models/bundles are supported, create an inference operator for each.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        # This is the inference operator for the spleen_model bundle. Note the model name.
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            bundle_config_names=config_names,
            model_name="spleen_ct",
        )

        # This is the inference operator for the pancreas_ct_dints bundle. Note the model name.
        bundle_pancreas_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            model_name="pancreas_ct_dints",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        #
        # NOTE: Each generated DICOM Seg will be a dcm file with the name based on the SOP instance UID.

        # Description for the Spleen seg, and the seg writer obj
        seg_descriptions_spleen = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags_spleen = {"SeriesDescription": "AI Spleen Seg for research use only. Not for clinical use."}
        dicom_seg_writer_spleen = DICOMSegmentationWriterOperator(
            segment_descriptions=seg_descriptions_spleen, custom_tags=custom_tags_spleen
        )

        # Description for the Pancreas seg, and the seg writer obj
        seg_descriptions_pancreas = [
            SegmentDescription(
                segment_label="Pancreas",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Pancreas,
                algorithm_name="volumetric (3D) segmentation of the pancreas from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.0",
            )
        ]
        custom_tags_pancreas = {"SeriesDescription": "AI Pancreas Seg for research use only. Not for clinical use."}

        dicom_seg_writer_pancreas = DICOMSegmentationWriterOperator(
            segment_descriptions=seg_descriptions_pancreas, custom_tags=custom_tags_pancreas
        )

        # NOTE: Sharp eyed readers can already see that the above instantiation of object can be simply parameterized.
        #       Very true, but leaving them as if for easy reading. In fact the whole app can be parameterized for general use.

        # Create the processing pipeline, by specifying the upstream and downstream operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )

        # Feed the input image to all inference operators
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # The Pancreas CT Seg bundle requires PyTorch 1.12.0 to avoid failure to load.
        self.add_flow(series_to_vol_op, bundle_pancreas_seg_op, {"image": "image"})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_spleen, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer_spleen, {"pred": "seg_image"})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_pancreas, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_pancreas_seg_op, dicom_seg_writer_pancreas, {"pred": "seg_image"})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    # Creates the app and test it standalone. When running is this mode, please note the following:
    #     -m <model file>, for model file path
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    # e.g.
    #     monai-deploy exec app.py -i input -m model/model.ts
    #
    logging.basicConfig(level=logging.DEBUG)
    app_instance = App(do_run=True)

INFO:root:Begin compose
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
INFO:root:End compose
INFO:__main__.App:Begin run
INFO:__main__.App:End run


## Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `dcm` and the Torch Script model at `model.ts`. Please use the actual path in your environment.


In [5]:
app = App()

app.run(input="dcm", output="output", model="multi_models")

INFO:root:Begin compose
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
INFO:root:End compose
INFO:__main__.App:Begin run


Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 213316, Operator ID: 88d4511f-740e-4790-b1c8-75d0cd026e42)


[2023-07-07 14:48:34,135] [INFO] (root) - Finding series for Selection named: CT Series
[2023-07-07 14:48:34,135] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2023-07-07 14:48:34,136] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2023-07-07 14:48:34,137] [INFO] (root) - On attribute: 'StudyDescription' to match value: '(.*?)'
[2023-07-07 14:48:34,137] [INFO] (root) -     Series attribute StudyDescription value: CT ABDOMEN W IV CONTRAST
[2023-07-07 14:48:34,138] [INFO] (root) - Series attribute string value did not match. Try regEx.
[2023-07-07 14:48:34,139] [INFO] (root) - On attribute: 'Modality' to match value: '(?i)CT'
[2023-07-07 14:48:34,139] [INFO] (root) -     Series attribute Modality value: CT
[2023-07-07 14:48:34,140] [INFO] (root) - Series attribute string value did not match. Try regEx.
[2023-07-07 14:48:34,141] [INFO] (root) - On att

Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 213316, Operator ID: 9e12794e-d30f-4a1e-96fc-2e966dc745e2)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 213316, Operator ID: 9608c853-02e6-4022-827d-54a800e31f51)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator MonaiBundleInferenceOperator
Executing operator MonaiBundleInferenceOperator (Process ID: 213316, Operator ID: 6965fd0a-0474-4b48-8195-b905f3981502)


/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary SaveImaged.__init__:resample: Current default value of argument `resample=True` has been deprecated since version 1.1. It will be changed to `resample=False` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


Done performing execution of operator MonaiBundleInferenceOperator

Going to initiate execution of operator MonaiBundleInferenceOperator
Executing operator MonaiBundleInferenceOperator (Process ID: 213316, Operator ID: 92dbc72f-274a-41e9-a64d-d965daac002f)
Done performing execution of operator MonaiBundleInferenceOperator

Going to initiate execution of operator DICOMSegmentationWriterOperator
Executing operator DICOMSegmentationWriterOperator (Process ID: 213316, Operator ID: f4880421-0076-41cd-8cba-e13871fa00cb)


/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/highdicom/valuerep.py:54: UserWarning: The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  warnings.warn(
[2023-07-07 14:50:12,070] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/pydicom/valuerep.py:443: UserWarning: A value of type 'int64' cannot be assigned to a tag with VR UL.
  warnings.warn(msg)
/home/mqin/src/monai-deploy-app-sdk/.venv05/lib/python3.8/site-packages/pydicom/valuerep.py

Done performing execution of operator DICOMSegmentationWriterOperator

Going to initiate execution of operator DICOMSegmentationWriterOperator
Executing operator DICOMSegmentationWriterOperator (Process ID: 213316, Operator ID: 74b80b84-160e-4d47-ad17-d2ab92b9789c)


[2023-07-07 14:50:14,318] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
[2023-07-07 14:50:14,320] [INFO] (highdicom.seg.sop) - add plane #1 for segment #1
[2023-07-07 14:50:14,321] [INFO] (highdicom.seg.sop) - add plane #2 for segment #1
[2023-07-07 14:50:14,322] [INFO] (highdicom.seg.sop) - add plane #3 for segment #1
[2023-07-07 14:50:14,323] [INFO] (highdicom.seg.sop) - add plane #4 for segment #1
[2023-07-07 14:50:14,325] [INFO] (highdicom.seg.sop) - add plane #5 for segment #1
[2023-07-07 14:50:14,326] [INFO] (highdicom.seg.sop) - add plane #6 for segment #1
[2023-07-07 14:50:14,327] [INFO] (highdicom.seg.sop) - add plane #7 for segment #1
[2023-07-07 14:50:14,328] [INFO] (highdicom.seg.sop) - add plane #8 for segment #1
[2023-07-07 14:50:14,330] [INFO] (highdicom.seg.sop) - add plane #9 for segment #1
[2023-07-07 14:50:14,331] [INFO] (highdicom.seg.sop) - add plane #10 for segment #1
[2023-07-07 14:50:14,332] [INFO] (highdicom.seg.sop) - add plane #11 for segment #1
[2

Done performing execution of operator DICOMSegmentationWriterOperator



Once the application is verified inside Jupyter notebook, we can write the above Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
my_app
├── __main__.py
└── app.py
```

In [6]:
# Create an application folder
!mkdir -p my_app

### app.py

In [7]:
%%writefile my_app/app.py

# Copyright 2021-2022 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

import monai.deploy.core as md
from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)


@resource(cpu=1, gpu=1, memory="7Gi")
# Enforcing torch>=1.12.0 because one of the Bundles/TorchScripts, Pancreas CT Seg, was created
# with this version, and would fail to jit.load with lower version of torch.
# The Bundle Inference Operator as of now only requires torch>=1.10.2, and does not yet dynamically
# parse the MONAI bundle to get the required pip package or ver on initialization, hence it does not set
# its own @env decorator accordingly when the app is being packaged into a MONAI Package.
@md.env(pip_packages=["torch>=1.12.0"])
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
# The monai pkg is not required by this class, instead by the included operators.
class App(Application):
    """This example demonstrates how to create a multi-model/multi-AI application.

    The important steps are:
        1. Place the model TorchScripts in a defined folder structure, see below for details
        2. Pass the model name to the inference operator instance in the app
        3. Connect the input to and output from the inference operators, as required by the app

    Model organization for this example:

        multi_models
        ├── pancreas_ct_dints
        │   └── model.ts
        └── spleen_ct
            └── model.ts

    Note:
    1. The TorchScript files of MONAI Bundles can be downloaded from MONAI Model Zoo, at
       https://github.com/Project-MONAI/model-zoo/tree/dev/models
    2. The input DICOM instances are from a DICOM Series of CT Abdomen, similar to the ones
       used in the Spleen Segmentation example
    3. This example is purely for technical demonstration, not for clinical use
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init to provide the optional packages info, parsed from the bundle, to the packager
        # for it to install the packages in the MAP docker image.
        # Setting output IOType to DISK only works only for leaf operators, not the case in this example.
        # When multiple models/bundles are supported, create an inference operator for each.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation, v0.3.2
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/pancreas_ct_dints_segmentation, v0.3

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        # This is the inference operator for the spleen_model bundle. Note the model name.
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            bundle_config_names=config_names,
            model_name="spleen_ct",
        )

        # This is the inference operator for the pancreas_ct_dints bundle. Note the model name.
        bundle_pancreas_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            model_name="pancreas_ct_dints",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        #
        # NOTE: Each generated DICOM Seg will be a dcm file with the name based on the SOP instance UID.

        # Description for the Spleen seg, and the seg writer obj
        seg_descriptions_spleen = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags_spleen = {"SeriesDescription": "AI Spleen Seg for research use only. Not for clinical use."}
        dicom_seg_writer_spleen = DICOMSegmentationWriterOperator(
            segment_descriptions=seg_descriptions_spleen, custom_tags=custom_tags_spleen
        )

        # Description for the Pancreas seg, and the seg writer obj
        seg_descriptions_pancreas = [
            SegmentDescription(
                segment_label="Pancreas",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Pancreas,
                algorithm_name="volumetric (3D) segmentation of the pancreas from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.0",
            )
        ]
        custom_tags_pancreas = {"SeriesDescription": "AI Pancreas Seg for research use only. Not for clinical use."}

        dicom_seg_writer_pancreas = DICOMSegmentationWriterOperator(
            segment_descriptions=seg_descriptions_pancreas, custom_tags=custom_tags_pancreas
        )

        # NOTE: Sharp eyed readers can already see that the above instantiation of object can be simply parameterized.
        #       Very true, but leaving them as if for easy reading. In fact the whole app can be parameterized for general use.

        # Create the processing pipeline, by specifying the upstream and downstream operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )

        # Feed the input image to all inference operators
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # The Pancreas CT Seg bundle requires PyTorch 1.12.0 to avoid failure to load.
        self.add_flow(series_to_vol_op, bundle_pancreas_seg_op, {"image": "image"})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_spleen, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer_spleen, {"pred": "seg_image"})

        # Create DICOM Seg for one of the inference output
        # Note below the dicom_seg_writer requires two inputs, each coming from a upstream operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer_pancreas, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_pancreas_seg_op, dicom_seg_writer_pancreas, {"pred": "seg_image"})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    # Creates the app and test it standalone. When running is this mode, please note the following:
    #     -m <model file>, for model file path
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    # e.g.
    #     monai-deploy exec app.py -i input -m model/model.ts
    #
    logging.basicConfig(level=logging.DEBUG)
    app_instance = App(do_run=True)


Overwriting my_app/app.py


```python
if __name__ == "__main__":
    App(do_run=True)
```

The above lines are needed to execute the application code by using `python` interpreter.

### \_\_main\_\_.py

\_\_main\_\_.py is needed for <a href="../../developing_with_sdk/packaging_app.html#required-arguments">MONAI Application Packager</a> to detect the main application code (`app.py`) when the application is executed with the application folder path (e.g., `python simple_imaging_app`).

In [8]:
%%writefile my_app/__main__.py
import logging
from app import App

if __name__ == "__main__":
    logging.basicConfig(level=logging.DEBUG)
    App(do_run=True)

Overwriting my_app/__main__.py


In [9]:
!ls my_app

app.py	__main__.py  __pycache__  spleen_seg_operator.py


In this time, let's execute the app in the command line.

In [10]:
!python my_app -i dcm -o output -m multi_models

INFO:root:Begin compose
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
INFO:root:End compose
INFO:app.App:Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 213660, Operator ID: cfef94ed-0213-4571-8412-148bb56fd16c)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 213660, Operator ID: 53e22d7b-adc9-42c3-a7ca-d1d5510f0661)
[2023-07-07 14:50:21,313] [INFO] (root) - Finding series for Selection named: CT Series
[2023-07-07 14:50:21,313] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2023-07-07 14:50:21,313] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.1194

Above command is same with the following command line:

In [11]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!monai-deploy exec my_app -i dcm -o output -m multi_models

INFO:root:Begin compose
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
INFO:root:End compose
INFO:app.App:Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 213753, Operator ID: 8be134c0-57ee-4298-a113-6d94da804902)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 213753, Operator ID: 85742313-fc70-45ee-b5cb-f65e47e8e651)
[2023-07-07 14:52:03,789] [INFO] (root) - Finding series for Selection named: CT Series
[2023-07-07 14:52:03,790] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2023-07-07 14:52:03,790] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.1194

In [12]:
!ls output

1.2.826.0.1.3680043.10.511.3.11231444010124776175218536511663543.dcm
1.2.826.0.1.3680043.10.511.3.11779019635288615017320261444184585.dcm
1.2.826.0.1.3680043.10.511.3.11927807764923034176934830784797996.dcm
1.2.826.0.1.3680043.10.511.3.26756833125967779911222791720326843.dcm
1.2.826.0.1.3680043.10.511.3.41468577108021529587933891221927885.dcm
1.2.826.0.1.3680043.10.511.3.94363520267823963721211927563964890.dcm


## Packaging app

Let's package the app with [MONAI Application Packager](/developing_with_sdk/packaging_app).

Note that with option `-b` a specific Nvidia PyTorch base docker image can be specified. Earlier versions of this image, e.g. `nvcr.io/nvidia/pytorch:21.11-py3`, caused Docker runtime errors in PyTorch, though the error only occurred when run in Jupyter.

In [13]:
!monai-deploy package -b nvcr.io/nvidia/pytorch:22.08-py3 my_app --tag my_app:latest -m multi_models

[2023-07-07 14:53:46,381] [INFO] (root) - Begin compose
[2023-07-07 14:53:46,386] [INFO] (root) - End compose
Building MONAI Application Package... \[+] Building 0.0s (0/0)                                                         
[?25|[+] Building 0.0s (0/0)                                                         
[?25/[+] Building 0.1s (4/19)                                                        
 => [internal] load build definition from dockerfile                       0.0s
 => => transferring dockerfile: 2.58kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 1.11kB                                        0.0s
 => [internal] load metadata for nvcr.io/nvidia/pytorch:22.08-py3          0.0s
 => [ 1/15] FROM nvcr.io/nvidia/pytorch:22.08-py3                          0.0s
 => [internal] load build context                                          0.1s
[+] Building 0.3s (4/19)            

:::{note}
Building a MONAI Application Package (Docker image) can take time. Use `-l DEBUG` option if you want to see the progress.
:::

We can see that the Docker image is created.

In [14]:
!docker image ls | grep my_app

my_app                                                                latest                                     b58b68c777ba   15 hours ago    15.6GB


## Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](/developing_with_sdk/executing_packaged_app_locally).

In [15]:
# Copy DICOM files are in 'dcm' folder

# Launch the app
!monai-deploy run my_app:latest dcm output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "my_app:latest" is available...

Checking for MAP "my_app:latest" locally
"my_app:latest" found.

Reading MONAI App Package manifest...
Preparing to copy...Copying from container - 0BSuccessfully copied 2.05kB to /tmp/tmpa6oazpry/app.json
Preparing to copy...Copying from container - 0BSuccessfully copied 2.56kB to /tmp/tmpa6oazpry/pkg.json
--> Verifying if "nvidia-docker" is installed...

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
INFO:root:Begin compose
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
DEBUG:root:Bundle path, None, not valid. Will get it in the execution context.
INFO:root:End compose
INFO:__main__.App:Begin run
Going to

In [16]:
!ls output

1.2.826.0.1.3680043.10.511.3.11231444010124776175218536511663543.dcm
1.2.826.0.1.3680043.10.511.3.11779019635288615017320261444184585.dcm
1.2.826.0.1.3680043.10.511.3.11927807764923034176934830784797996.dcm
1.2.826.0.1.3680043.10.511.3.26756833125967779911222791720326843.dcm
1.2.826.0.1.3680043.10.511.3.26951083066705886522839047001116455.dcm
1.2.826.0.1.3680043.10.511.3.41468577108021529587933891221927885.dcm
1.2.826.0.1.3680043.10.511.3.60922799005047068192048147966405053.dcm
1.2.826.0.1.3680043.10.511.3.94363520267823963721211927563964890.dcm
